In [66]:
import pytesseract
import cv2
from PIL import Image
import numpy as np
import re
import config
import clr
import os
clr.AddReference(os.path.abspath(config.FIT_DLL))
from Dynastream.Fit import * # type: ignore

pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract'

from Swimming import SwimmingWorkout, SwimmingWorkoutStepRepeat, SwimmingWorkoutStepSwim, SwimmingWorkoutStepSwimRest

In [73]:
image_bgr = cv2.imread("images/IMG_9247.PNG")
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
image_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)

# Define blue range in HSV
lower_blue = np.array([90, 50, 50])
upper_blue = np.array([130, 255, 255])
mask = cv2.inRange(image_hsv, lower_blue, upper_blue)

instructions = []

contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for cnt in contours[::-1]:
    x, y, w, h = cv2.boundingRect(cnt)
    if w > 100 and w < 200 and h > 100:  # Filter out small noise
        # Crop a horizontal strip to the right of the blue circle
        line_crop = image_rgb[y:y+h, x+w:x+w+1000] # Adjust width as needed

        # Convert to PIL Image for pytesseract
        pil_img = Image.fromarray(line_crop)

        # display(pil_img) # for debugging

        text = pytesseract.image_to_string(pil_img)

        instructions.append(text.replace('\n', '').replace(' ', '').strip())

In [76]:
message_index = 0
for instruction in instructions:
    pattern_repeat = re.search(r'(\d+)\s*[x×]\s*\((.*?)\)', instruction, re.IGNORECASE)
    if pattern_repeat:
        times = int(pattern_repeat.group(1))
        tasks = str(pattern_repeat.group(2).strip()).split(';')
        print(f"Repeat {times} times: {task}")

        for task in tasks:
            print(task)
            pattern_distance = re.search(r'(\d+)\s*m\s*(.*)', task, re.IGNORECASE)
            if pattern_distance:
                distance = int(pattern_distance.group(1))
                swim_type = pattern_distance.group(2).strip().lower()
                print(distance, swim_type)
                SwimmingWorkoutStepSwim.create_workout_step_swim(
                    message_index= message_index, 
                    distance= distance,
                    swim_stroke= SwimStroke.Invalid # type: ignore
                )
                message_index += 1

        # SwimmingWorkoutStepRepeat(
        #     message_index= message_index,
        #     repeat_from: int, 
        #     repetitions: int
        # )

Repeat 6 times: 25mcrawlrattrapé
25mcrawlrattrapé
25 crawlrattrapé
25mcrawl
25 crawl
15srécup
